<a href="https://colab.research.google.com/github/christianwarmuth/transformer_adapter_bias_evaluation/blob/main/colab/Adapter_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Evaluating Adapter 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)
#drive.mount("/content/gdrive/MyDrive/master_hpi/NLP_Project/code/", force_remount=True)

Mounted at /content/gdrive/


In [3]:
import sys
sys.path.append('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/')



In [4]:
path = "/content/gdrive/MyDrive/master_hpi/NLP_Project/code/"

In [5]:
!pip install transformers==3.0.2
!pip3 install -r /content/gdrive/MyDrive/master_hpi/NLP_Project/code/requirements.txt

  Using cached https://files.pythonhosted.org/packages/27/3c/91ed8f5c4e7ef3227b4119200fc0ed4b4fd965b1f0172021c25701087825/transformers-3.0.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/02/59/68c7e3833f535615fb97d33ffcb7b30bbf62bc7477a9c59cd19ad8535d72/tokenizers-0.8.1rc1-cp37-cp37m-manylinux1_x86_64.whl
ERROR: adapter-transformers 2.0.1 has requirement tokenizers<0.11,>=0.10.1, but you'll have tokenizers 0.8.1rc1 which is incompatible.
  Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Found existing installation: transformers 2.7.0
    Uninstalling transformers-2.7.0:
      Successfully uninstalled transformers-2.7.0
  Using cached https://files.pythonhosted.org/packages/d6/e3/5e49e9a83fb605aaa34a1c1173e607302fecae529428c28696fb18f1c2c9/tokenizers-0.5.2-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/37/ba/dda44bbf35b071441635708a3dd5

In [6]:
!pip install adapter-transformers
!pip install datasets
!pip install colorama

  Using cached https://files.pythonhosted.org/packages/d4/e2/df3543e8ffdab68f5acc73f613de9c2b155ac47f162e725dcac87c521c11/tokenizers-0.10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl
ERROR: transformers 2.7.0 has requirement tokenizers==0.5.2, but you'll have tokenizers 0.10.3 which is incompatible.
  Found existing installation: tokenizers 0.5.2
    Uninstalling tokenizers-0.5.2:
      Successfully uninstalled tokenizers-0.5.2


In [7]:
import pandas as pd
import dataloader
from collections import defaultdict 

# Dataset Loading - Stereoset

In [8]:

print(path+"dev.json")

/content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json


In [9]:
stereoset = dataloader.StereoSet(path+"dev.json")

In [10]:
stereoset

In [11]:
intersentence_examples = stereoset.get_intersentence_examples() 
intrasentence_examples = stereoset.get_intrasentence_examples()

In [12]:
intrasentence_examples[1].sentences[1].gold_label

'anti-stereotype'

In [13]:
id2term = {}
id2gold = {}
id2score = {}
example2sent = {}
domain2example = {"intersentence": defaultdict(lambda: []), "intrasentence": defaultdict(lambda: [])}

for example in intrasentence_examples:
  for sentence in example.sentences:
    id2term[sentence.ID] = example.target
    id2gold[sentence.ID] = sentence.gold_label
    example2sent[(example.ID, sentence.gold_label)] = sentence.ID
    domain2example['intrasentence'][example.bias_type].append(example)

for example in intersentence_examples:
  for sentence in example.sentences:
    id2term[sentence.ID] = example.target
    id2gold[sentence.ID] = sentence.gold_label
    example2sent[(example.ID, sentence.gold_label)] = sentence.ID
    domain2example['intersentence'][example.bias_type].append(example)

In [14]:
# When predictions are already done
#for sent in self.predictions.get('intrasentence', []) + self.predictions.get('intersentence', []):
#            self.id2score[sent['id']] = sent['score']

In [15]:
domain2example

{'intersentence': defaultdict(<function __main__.<lambda>>,
             {'gender': [<dataloader.IntersentenceExample at 0x7f51df60c0d0>,
              'profession': [<dataloader.IntersentenceExample at 0x7f51df60f1d0>,
               ...],
              'race': [<dataloader.IntersentenceExample at 0x7f51df6095d0>,
               ...],
              'religion': [<dataloader.IntersentenceExample at 0x7f51df423890>,
               <dataloader.IntersentenceExample at 0x7f51dedaf5d0>]}),
 'intrasentence': defaultdict(<function __main__.<lambda>>,
             {'gender': [<dataloader.IntrasentenceExample at 0x7f51dff14d50>,
              'profession': [<dataloader.IntrasentenceExample at 0x7f51dff0cd10>,
               ...],
              'race': [<dataloader.IntrasentenceExample at 0x7f51dff12550>,
               ...],
              'religion': [<dataloader.IntrasentenceExample at 0x7f51dfd3dbd0>,
               <dataloader.IntrasentenceExample at 0x7f51df669f90>]})}

In [16]:
!python3 -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [17]:
from eval_template import *
import dataloader


class Args:
  no_cuda = False
  input_file = '/content/dev.json'
  output_file = '/content/predictions.json'
  max_seq_length = 128
  batch_size = 1

args=Args()

#class input_args:
#  def __init__(self, no_cuda, input_file, output_file, max_seq_length):
#    self.no_cuda = no_cuda
 #   self.input_file = input_file
 #   self.output_file = output_file
#    self.max_seq_length = max_seq_length

#args = input_args(False, "/content/dev.json", "/content/predictions.json", 128)
#args = {#
#    "no_cuda": False, 
#    "input_file": "/content/dev.json", 
#    "output_file":"/content/predictions.json"
#}

evaluator = BiasEvaluator(**vars(args))
results = evaluator.evaluate()

if args.output_file is not None:
  output_file = args.output_file
else:
  output_file = f"predictions_SentimentModel.json"

  output_file = os.path.join(args.output_dir, output_file)
with open(output_file, "w+") as f:
  json.dump(results, f, indent=2)


ImportError: ignored

In [ ]:
!pip3 install numpy


In [ ]:
#!wget -P pretrained_models http://moinnadeem.com/stereoset/pretrained_models/SentimentBert.pth


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
class Args:
  no_cuda = False
  input_file = '/content/sample_data/dev.json'
  output_file = '/content/sample_data/predictions.json'

args=Args()

In [ ]:
print(vars(args))